# Debezium Lakehouse Integration

This notebook demonstrates how to query data captured by Debezium and stored in Apache Iceberg tables.

## Query Customers Table

The Spark session is already configured to use the REST Catalog, so we can directly query the tables created by the Iceberg Connect Sink.

In [ ]:
spark.sql("SELECT * FROM my_database.customers_table LIMIT 10").show()

We can use the Jupyter magic to make the query too:

In [ ]:
%%sql

SELECT * FROM my_database.customers_table LIMIT 10

## Query Products Table

Now let's query the products table:

In [ ]:
%%sql

SELECT * FROM my_database.products_table LIMIT 10

## Query Orders Table

Let's examine the orders table:

In [ ]:
%%sql

SELECT * FROM my_database.orders_table LIMIT 10

## Query Order Items Table

Finally, let's look at the order items table:

In [ ]:
%%sql

SELECT * FROM my_database.order_items_table LIMIT 10

## Join Tables for Analysis

Now let's perform a more complex query joining multiple tables:

In [ ]:
%%sql

SELECT 
    c.id as customer_id, 
    c.first_name, 
    c.last_name, 
    o.id as order_id,
    o.order_date,
    p.name as product_name,
    oi.quantity,
    oi.unit_price,
    (oi.quantity * oi.unit_price) as total_price
FROM 
    my_database.customers_table c
JOIN 
    my_database.orders_table o ON o.customer_id = c.id
JOIN 
    my_database.order_items_table oi ON oi.order_id = o.id
JOIN 
    my_database.products_table p ON p.id = oi.product_id
LIMIT 10

## Analyze Customer Spending

Let's calculate total spending by customer:

In [ ]:
%%sql

SELECT 
    c.first_name || ' ' || c.last_name as customer_name,
    COUNT(DISTINCT o.id) as order_count,
    SUM(oi.quantity * oi.unit_price) as total_spent
FROM 
    my_database.customers_table c
JOIN 
    my_database.orders_table o ON o.customer_id = c.id
JOIN 
    my_database.order_items_table oi ON oi.order_id = o.id
GROUP BY 
    c.first_name, c.last_name
ORDER BY 
    total_spent DESC
LIMIT 10